# 패키지 업로드

In [2]:
!pip install transformers
!pip install rouge
!pip install rouge-score

     |████████████████████████████████| 2.1MB 4.8MB/s 
     |████████████████████████████████| 3.3MB 35.3MB/s 
     |████████████████████████████████| 901kB 49.0MB/s 


# 데이터 살펴보기 및 전처리

In [3]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 기술과 응용/중간고사/articles1.csv')
data

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...,...,...,...,...,...,...
49995,53287,73465,"Rex Tillerson Says Climate Change Is Real, but …",Atlantic,Robinson Meyer,2017-01-11,2017.0,1.0,NaN,"As chairman and CEO of ExxonMobil, Rex Tillers..."
49996,53288,73466,The Biggest Intelligence Questions Raised by t...,Atlantic,Amy Zegart,2017-01-11,2017.0,1.0,NaN,I’ve spent nearly 20 years looking at intellig...
49997,53289,73467,Trump Announces Plan That Does Little to Resol...,Atlantic,Jeremy Venook,2017-01-11,2017.0,1.0,NaN,Donald Trump will not be taking necessary st...
49998,53290,73468,Dozens of For-Profit Colleges Could Soon Close,Atlantic,Emily DeRuy,2017-01-11,2017.0,1.0,NaN,Dozens of colleges could be forced to close ...


In [6]:
data['publication'].value_counts()

Breitbart           23781
CNN                 11488
New York Times       7803
Business Insider     6757
Atlantic              171
Name: publication, dtype: int64

In [7]:
data['year'].value_counts()

2016.0    28451
2017.0    17908
2015.0     3326
2013.0      212
2014.0       76
2012.0       26
2011.0        1
Name: year, dtype: int64

In [8]:
# 요약할 때 길이가 너무 긴 문장들 제외.
b = []
for i in range(len(data)):
  if len(data.iloc[i]['content']) <= 3000:
    b.append(i)

data = data.iloc[b]
data

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
5,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017.0,1.0,NaN,"LONDON — Queen Elizabeth II, who has been b..."
19,19,17303,Fecal Pollution Taints Water at Melbourne’s Be...,New York Times,Brett Cole,2017-01-03,2017.0,1.0,NaN,"SYDNEY, Australia — The annual beach pilgri..."
33,33,17323,"Airline Pilot, Believed to Be Drunk, Is Pulled...",New York Times,Ian Austen,2017-01-03,2017.0,1.0,NaN,OTTAWA — It was 7 a. m. and 99 passengers a...
47,47,17338,It’s Time to Ignore Advice About Which Stocks ...,New York Times,Damon Darlin,2017-01-09,2017.0,1.0,NaN,It’s that time of year when financial advice s...
52,52,17344,Chinese City Official Shoots 2 Others and Kill...,New York Times,Chris Buckley and Adam Wu,2017-01-05,2017.0,1.0,NaN,BEIJING — A city official in southwest Chin...
...,...,...,...,...,...,...,...,...,...,...
49905,53196,73320,The Hacking Investigation Trump Wants Instead,Atlantic,Kaveh Waddell,2017-01-06,2017.0,1.0,NaN,Donald Trump wants to get to the bottom of n...
49971,53263,73441,Michael Cohen: ’It Is Fake News Meant to Malig...,Atlantic,Rosie Gray,2017-01-10,2017.0,1.0,NaN,Donald Trump and his lawyer on Tuesday night d...
49976,53268,73446,The U.S. Supreme Court Puts North Carolina’s 2...,Atlantic,David A. Graham,2017-01-10,2017.0,1.0,NaN,"DURHAM, N. C. — The Supreme Court has a messa..."
49983,53275,73453,‘We Have a Problem’: John Kerry on Making Poli...,Atlantic,Uri Friedman,2017-01-10,2017.0,1.0,NaN,In one of his last public appearances as U. S....


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# 개수가 너무 많기에 임의로 1%만 추출하여 263개의 데이터에 대해 요약을 진행해 볼 예정.
train, test = train_test_split(data, test_size=0.01, random_state=42)

In [11]:
test

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
17198,17199,35938,First Look: Woody Allen-Miley Cyrus Series ’Cr...,Breitbart,Daniel Nussbaum,2016-09-14,2016.0,9.0,NaN,Amazon released the first trailer for its upco...
22470,22473,41220,Comedian Sacha Baron Cohen Suggests Trump will...,Breitbart,Kipp Jones,2016-03-04,2016.0,3.0,NaN,British satirist and comedian Sacha Baron Cohe...
37279,38374,57218,Southwest flight makes emergency landing,CNN,NaN,2016-08-28,2016.0,8.0,NaN,(CNN) A Southwest Airlines flight from New Or...
20955,20958,39704,Bolton: Trump’s Taiwan Call ’Absolutely Not a ...,Breitbart,Dan Riehl,2016-12-09,2016.0,12.0,NaN,Asked about Donald Trump’s recent phone call...
35547,36329,55153,"Gunmen attack Mali luxury resort, at least 2 dead",CNN,NaN,2017-06-18,2017.0,6.0,NaN,(CNN) At least two tourists were killed and 3...
...,...,...,...,...,...,...,...,...,...,...
17778,17780,36520,Watch: Rove in Heated Exchange While Defending...,Breitbart,Pam Key,2016-04-01,2016.0,4.0,NaN,"Friday on Fox News Channel’s “Your World,” Rep..."
42919,45617,64600,Forty years of self-portraits,CNN,NaN,2015-07-14,2015.0,7.0,NaN,(CNN) The excuses for a photographer to take ...
46159,49162,68617,We just got a new insight into a crucial crisi...,Business Insider,Matt Turner,2016-03-12,2016.0,3.0,NaN,"’’ ’’ ’’ ’’ ’’ ’’ ’’ ’’ ’’ ’In short, Goldman’..."
10608,10608,29344,Mexican Border State Cops Allegedly Beat Innoc...,Breitbart,Cartel Chronicles,2017-02-19,2017.0,2.0,NaN,"PIEDRAS NEGRAS, Coahuila — Mexican authorit..."


In [12]:
# 제목과 기사내용만 필요.
test = test[['title', 'content']]
test

,title,content
17198,First Look: Woody Allen-Miley Cyrus Series ’Cr...,Amazon released the first trailer for its upco...
22470,Comedian Sacha Baron Cohen Suggests Trump will...,British satirist and comedian Sacha Baron Cohe...
37279,Southwest flight makes emergency landing,(CNN) A Southwest Airlines flight from New Or...
20955,Bolton: Trump’s Taiwan Call ’Absolutely Not a ...,Asked about Donald Trump’s recent phone call...
35547,"Gunmen attack Mali luxury resort, at least 2 dead",(CNN) At least two tourists were killed and 3...
...,...,...
17778,Watch: Rove in Heated Exchange While Defending...,"Friday on Fox News Channel’s “Your World,” Rep..."
42919,Forty years of self-portraits,(CNN) The excuses for a photographer to take ...
46159,We just got a new insight into a crucial crisi...,"’’ ’’ ’’ ’’ ’’ ’’ ’’ ’’ ’’ ’In short, Goldman’..."
10608,Mexican Border State Cops Allegedly Beat Innoc...,"PIEDRAS NEGRAS, Coahuila — Mexican authorit..."


In [13]:
# 쓸데 없는 기호들 빼기.
for i in range(len(test)):
  test['title'].iloc[i] = test['title'].iloc[i].strip()
  test['content'].iloc[i] = test['content'].iloc[i].replace('’', '').strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [14]:
test

,title,content
17198,First Look: Woody Allen-Miley Cyrus Series ’Cr...,Amazon released the first trailer for its upco...
22470,Comedian Sacha Baron Cohen Suggests Trump will...,British satirist and comedian Sacha Baron Cohe...
37279,Southwest flight makes emergency landing,(CNN) A Southwest Airlines flight from New Orl...
20955,Bolton: Trump’s Taiwan Call ’Absolutely Not a ...,Asked about Donald Trumps recent phone call ...
35547,"Gunmen attack Mali luxury resort, at least 2 dead",(CNN) At least two tourists were killed and 32...
...,...,...
17778,Watch: Rove in Heated Exchange While Defending...,"Friday on Fox News Channels “Your World,” Repu..."
42919,Forty years of self-portraits,(CNN) The excuses for a photographer to take a...
46159,We just got a new insight into a crucial crisi...,"In short, Goldmans early collateral calls kept..."
10608,Mexican Border State Cops Allegedly Beat Innoc...,"PIEDRAS NEGRAS, Coahuila — Mexican authorit..."


# 모형 및 학습

### pipeline summary

In [15]:
from transformers import pipeline

In [16]:
summ = pipeline("summarization")

In [17]:
# 제목보다는 길고, 간단하게 요약하기 위해 최소, 최대 길이의 parameter을 줌.
test['summary'] = 0
for i in range(len(test)):
  test['summary'].iloc[i] = summ(test.iloc[i]['content'], min_length=5, max_length=30)[0]['summary_text'].strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afte

In [18]:
test

,title,content,summary
17198,First Look: Woody Allen-Miley Cyrus Series ’Cr...,Amazon released the first trailer for its upco...,Amazon has released the first trailer for its ...
22470,Comedian Sacha Baron Cohen Suggests Trump will...,British satirist and comedian Sacha Baron Cohe...,Sacha Baron Cohen mocked Donald Trump at the U...
37279,Southwest flight makes emergency landing,(CNN) A Southwest Airlines flight from New Orl...,A woman who was on the plane with her husband ...
20955,Bolton: Trump’s Taiwan Call ’Absolutely Not a ...,Asked about Donald Trumps recent phone call ...,Donald Trump's recent phone call with Taiwan c...
35547,"Gunmen attack Mali luxury resort, at least 2 dead",(CNN) At least two tourists were killed and 32...,"Two of the three attackers were also killed, a..."
...,...,...,...
17778,Watch: Rove in Heated Exchange While Defending...,"Friday on Fox News Channels “Your World,” Repu...",Karl Rove got into a heated back and forth wit...
42919,Forty years of self-portraits,(CNN) The excuses for a photographer to take a...,Abby Robinsons collection ”AutoWorks” challeng...
46159,We just got a new insight into a crucial crisi...,"In short, Goldmans early collateral calls kept...",Cassano thinks that ”no one could get a handle...
10608,Mexican Border State Cops Allegedly Beat Innoc...,"PIEDRAS NEGRAS, Coahuila — Mexican authorit...",Mexican authorities are investigating some mem...


### T5 transformer summary

In [19]:
from transformers import AutoModelWithLMHead, AutoTokenizer

model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [20]:
test['summary_T5'] = 0
for i in range(len(test)):
  inputs = tokenizer.encode("summarize: " + test.iloc[i]['content'], return_tensors="pt", max_length=512)
  outputs = model.generate(inputs, max_length=30, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
  test['summary_T5'].iloc[i] = tokenizer.decode(outputs[0]).replace('<pad>', '').strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [21]:
test

,title,content,summary,summary_T5
17198,First Look: Woody Allen-Miley Cyrus Series ’Cr...,Amazon released the first trailer for its upco...,Amazon has released the first trailer for its ...,the series begins streaming on amazon prime on...
22470,Comedian Sacha Baron Cohen Suggests Trump will...,British satirist and comedian Sacha Baron Cohe...,Sacha Baron Cohen mocked Donald Trump at the U...,satirist and comedian Sacha Baron Cohen mocked...
37279,Southwest flight makes emergency landing,(CNN) A Southwest Airlines flight from New Orl...,A woman who was on the plane with her husband ...,a flight from new orleans to florida was force...
20955,Bolton: Trump’s Taiwan Call ’Absolutely Not a ...,Asked about Donald Trumps recent phone call ...,Donald Trump's recent phone call with Taiwan c...,former u.n. ambassador says he thought he was ...
35547,"Gunmen attack Mali luxury resort, at least 2 dead",(CNN) At least two tourists were killed and 32...,"Two of the three attackers were also killed, a...","two tourists killed, 32 others rescued after g..."
...,...,...,...,...
17778,Watch: Rove in Heated Exchange While Defending...,"Friday on Fox News Channels “Your World,” Repu...",Karl Rove got into a heated back and forth wit...,a guest host attempted to blame Rove for the l...
42919,Forty years of self-portraits,(CNN) The excuses for a photographer to take a...,Abby Robinsons collection ”AutoWorks” challeng...,abby Robinson's collection 'AutoWorks' is not ...
46159,We just got a new insight into a crucial crisi...,"In short, Goldmans early collateral calls kept...",Cassano thinks that ”no one could get a handle...,"Cassano: early collateral calls kept changing,..."
10608,Mexican Border State Cops Allegedly Beat Innoc...,"PIEDRAS NEGRAS, Coahuila — Mexican authorit...",Mexican authorities are investigating some mem...,authorities are investigating members of the F...


# parameter 조정

In [22]:
# length_penalty, num_beams 변경, repetition_penalty 추가

test['summary_T5_tuning'] = 0
for i in range(len(test)):
  inputs = tokenizer.encode("summarize: " + test.iloc[i]['content'], return_tensors="pt", max_length=512)
  outputs = model.generate(inputs, max_length=30, min_length=5, length_penalty=1.5, num_beams=5, repetition_penalty=2.5, early_stopping=True)
  test['summary_T5_tuning'].iloc[i] = tokenizer.decode(outputs[0]).replace('<pad>', '').strip()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [23]:
test

,title,content,summary,summary_T5,summary_T5_tuning
17198,First Look: Woody Allen-Miley Cyrus Series ’Cr...,Amazon released the first trailer for its upco...,Amazon has released the first trailer for its ...,the series begins streaming on amazon prime on...,the series begins streaming on amazon prime on...
22470,Comedian Sacha Baron Cohen Suggests Trump will...,British satirist and comedian Sacha Baron Cohe...,Sacha Baron Cohen mocked Donald Trump at the U...,satirist and comedian Sacha Baron Cohen mocked...,baron Cohen appeared on the red carpet wearing...
37279,Southwest flight makes emergency landing,(CNN) A Southwest Airlines flight from New Orl...,A woman who was on the plane with her husband ...,a flight from new orleans to florida was force...,a flight from new orleans to Orlando was force...
20955,Bolton: Trump’s Taiwan Call ’Absolutely Not a ...,Asked about Donald Trumps recent phone call ...,Donald Trump's recent phone call with Taiwan c...,former u.n. ambassador says he thought he was ...,former u.n. ambassador John Bolton says the ca...
35547,"Gunmen attack Mali luxury resort, at least 2 dead",(CNN) At least two tourists were killed and 32...,"Two of the three attackers were also killed, a...","two tourists killed, 32 others rescued after g...","two of the three attackers were also killed, a..."
...,...,...,...,...,...
17778,Watch: Rove in Heated Exchange While Defending...,"Friday on Fox News Channels “Your World,” Repu...",Karl Rove got into a heated back and forth wit...,a guest host attempted to blame Rove for the l...,a guest host attempted to blame Rove for the l...
42919,Forty years of self-portraits,(CNN) The excuses for a photographer to take a...,Abby Robinsons collection ”AutoWorks” challeng...,abby Robinson's collection 'AutoWorks' is not ...,abby Robinsons collection 'AutoWorks' is not a...
46159,We just got a new insight into a crucial crisi...,"In short, Goldmans early collateral calls kept...",Cassano thinks that ”no one could get a handle...,"Cassano: early collateral calls kept changing,...","Cassano: early collateral calls kept changing,..."
10608,Mexican Border State Cops Allegedly Beat Innoc...,"PIEDRAS NEGRAS, Coahuila — Mexican authorit...",Mexican authorities are investigating some mem...,authorities are investigating members of the F...,authorities are investigating members of the F...


# 평가

In [50]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [51]:
test['fmeasure_summary_rouge1'] = 0
test['fmeasure_summary_rougeL'] = 0

for i in range(len(test)):
  scores = scorer.score(test['title'].iloc[i], test['summary'].iloc[i])
  test['fmeasure_summary_rouge1'].iloc[i] = scores['rouge1'][2]
  test['fmeasure_summary_rougeL'].iloc[i] = scores['rougeL'][2]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [52]:
test['fmeasure_summary_T5_rouge1'] = 0
test['fmeasure_summary_T5_rougeL'] = 0

for i in range(len(test)):
  scores = scorer.score(test['title'].iloc[i], test['summary_T5'].iloc[i])
  test['fmeasure_summary_T5_rouge1'].iloc[i] = scores['rouge1'][2]
  test['fmeasure_summary_T5_rougeL'].iloc[i] = scores['rougeL'][2]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [53]:
test['fmeasure_summary_T5_tuning_rouge1'] = 0
test['fmeasure_summary_T5_tuning_rougeL'] = 0

for i in range(len(test)):
  scores = scorer.score(test['title'].iloc[i], test['summary_T5_tuning'].iloc[i])
  test['fmeasure_summary_T5_tuning_rouge1'].iloc[i] = scores['rouge1'][2]
  test['fmeasure_summary_T5_tuning_rougeL'].iloc[i] = scores['rougeL'][2]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [54]:
test

,title,content,summary,summary_T5,summary_T5_tuning,fmeasure_summary_rouge1,fmeasure_summary_rougeL,fmeasure_summary_T5_rouge1,fmeasure_summary_T5_rougeL,fmeasure_summary_T5_tuning_rouge1,fmeasure_summary_T5_tuning_rougeL
17198,First Look: Woody Allen-Miley Cyrus Series ’Cr...,Amazon released the first trailer for its upco...,Amazon has released the first trailer for its ...,the series begins streaming on amazon prime on...,the series begins streaming on amazon prime on...,0.444444,0.333333,0.121212,0.060606,0.121212,0.060606
22470,Comedian Sacha Baron Cohen Suggests Trump will...,British satirist and comedian Sacha Baron Cohe...,Sacha Baron Cohen mocked Donald Trump at the U...,satirist and comedian Sacha Baron Cohen mocked...,baron Cohen appeared on the red carpet wearing...,0.235294,0.235294,0.266667,0.266667,0.176471,0.176471
37279,Southwest flight makes emergency landing,(CNN) A Southwest Airlines flight from New Orl...,A woman who was on the plane with her husband ...,a flight from new orleans to florida was force...,a flight from new orleans to Orlando was force...,0.000000,0.000000,0.307692,0.307692,0.285714,0.285714
20955,Bolton: Trump’s Taiwan Call ’Absolutely Not a ...,Asked about Donald Trumps recent phone call ...,Donald Trump's recent phone call with Taiwan c...,former u.n. ambassador says he thought he was ...,former u.n. ambassador John Bolton says the ca...,0.300000,0.200000,0.117647,0.117647,0.242424,0.242424
35547,"Gunmen attack Mali luxury resort, at least 2 dead",(CNN) At least two tourists were killed and 32...,"Two of the three attackers were also killed, a...","two tourists killed, 32 others rescued after g...","two of the three attackers were also killed, a...",0.060606,0.060606,0.344828,0.275862,0.066667,0.066667
...,...,...,...,...,...,...,...,...,...,...,...
17778,Watch: Rove in Heated Exchange While Defending...,"Friday on Fox News Channels “Your World,” Repu...",Karl Rove got into a heated back and forth wit...,a guest host attempted to blame Rove for the l...,a guest host attempted to blame Rove for the l...,0.114286,0.114286,0.058824,0.058824,0.066667,0.066667
42919,Forty years of self-portraits,(CNN) The excuses for a photographer to take a...,Abby Robinsons collection ”AutoWorks” challeng...,abby Robinson's collection 'AutoWorks' is not ...,abby Robinsons collection 'AutoWorks' is not a...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
46159,We just got a new insight into a crucial crisi...,"In short, Goldmans early collateral calls kept...",Cassano thinks that ”no one could get a handle...,"Cassano: early collateral calls kept changing,...","Cassano: early collateral calls kept changing,...",0.111111,0.111111,0.055556,0.055556,0.055556,0.055556
10608,Mexican Border State Cops Allegedly Beat Innoc...,"PIEDRAS NEGRAS, Coahuila — Mexican authorit...",Mexican authorities are investigating some mem...,authorities are investigating members of the F...,authorities are investigating members of the F...,0.193548,0.193548,0.125000,0.125000,0.125000,0.125000


In [56]:
test['fmeasure_summary_rouge1'].mean()

0.283467635666216

In [57]:
test['fmeasure_summary_T5_rouge1'].mean()

0.23487093570609016

In [58]:
test['fmeasure_summary_T5_tuning_rouge1'].mean()

0.22915750335488638

In [59]:
test['fmeasure_summary_rougeL'].mean()

0.24341687627458483

In [60]:
test['fmeasure_summary_T5_rougeL'].mean()

0.204731627093903

In [61]:
test['fmeasure_summary_T5_tuning_rougeL'].mean()

0.19637867398432918

In [5]:
a = {'fmeasure_summary_rouge1':  [test['fmeasure_summary_rouge1'].mean()],
           'fmeasure_summary_T5_rouge1':  [test['fmeasure_summary_T5_rouge1'].mean()],
           'fmeasure_summary_T5_tuning_rouge1' : [test['fmeasure_summary_T5_tuning_rouge1'].mean()],
           'fmeasure_summary_rougeL': [test['fmeasure_summary_rougeL'].mean()], 
     'fmeasure_summary_T5_rougeL': [test['fmeasure_summary_T5_rougeL'].mean()], 
     'fmeasure_summary_T5_tuning_rougeL': [test['fmeasure_summary_T5_tuning_rougeL'].mean()]}

In [6]:
pd.DataFrame(a)

,fmeasure_summary_rouge1,fmeasure_summary_T5_rouge1,fmeasure_summary_T5_tuning_rouge1,fmeasure_summary_rougeL,fmeasure_summary_T5_rougeL,fmeasure_summary_T5_tuning_rougeL
0,0.283468,0.234871,0.229158,0.243417,0.204732,0.196379
